# Week 6  - Trees


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

This week we'll implement the Sankoff algorithm as seen in the Week5B lecture. This is a good introduction to programming with phylogenetic trees.
    
</div>

## Building Trees

### Creating the substitution matrix
First, we will create the $c_{ij}$ matrix that contains the substitution scores. As we saw in lecture, the score for no change is 0, the score for a trasition is 1.0 and the score for a transversion is 2.5.

We could hard code this since it's only 4x4 but here is some code that will build the matrix as a dictionary of dictionaries.

In [ ]:
purines = ['A', 'G']
pyrimidines = ['C', 'T']
CHARACTERS = purines + pyrimidines

c = dict()
for nt1 in CHARACTERS:
    c[nt1] = dict()
    for nt2 in CHARACTERS:
        # no change
        if nt1 == nt2:
            c[nt1][nt2] = 0.0
        # transition
        elif (nt1 in purines and nt2 in purines) or (nt1 in pyrimidines and nt2 in pyrimidines):
            c[nt1][nt2] = 1.0
        # transversion
        else:
            c[nt1][nt2] = 2.5

As you can see below, the dictionary has nucleotides as keys and another dictionary as values. This second level of dictionaries has nucleotides as keys and the $c_{ij}$ entries as values.

In [ ]:
# Let's view our substitution matrix
c

In practical terms, this allows us to easily get a specific $c_{ij}$ value using dictionary lookups, which improves code readability for minimal performance cost.

In [ ]:
c['A']['G']

### Initializing the tree
We will also implement the tree as a dictionary of dictionaries. The tree dictionary keys are node names and the values are node dictionaries. The node dictionaries have the node's data, scores and left/right node names, as the key-value pairs.

The following function creates an empty tree dictionary suitable for running the Sankoff algorithm on DNA data with the correct number of nodes given a number of tips.

In [ ]:
def init_tree(num_tips):
    """
    Create an empty tree dictionary with the correct number of nodes.
    This function does not link the nodes together, so there is no tree structure yet.
    """
    tree = dict()
    
    # create all the nodes
    tree['root'] = dict()
    for x in range(1, num_tips + 1):
        tree['tip{}'.format(x)] = dict()
    for x in range(1, num_tips + 1 - 2):
        tree['int{}'.format(x)] = dict()
    
    # initialize node data
    for node in tree:
        for child in ['left', 'right']:
            tree[node][child] = None
        for nt in CHARACTERS:
            tree[node][nt] = None
    
    return tree

In [ ]:
# Test the init_tree() function


### Exercise 1: Initializing tip values

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

<b>Challange:</b> The following function should initialize a tip node for the Sankoff algorithm to 0 for the observed nucleotide and infinity (`inf`) for others. Try to complete it. Note that the function modifies the tree in-place and does not return a value.

- [ ] Intialise tip nodes with cost of zero
- [ ] Internal nodes initialise with cost of infinity

</div>

In [ ]:
from math import inf

def init_tip(tree, tip_name, observed_nt):
    """
    Intialise a tip of a tree with the correct costs.
    At this tip node, only the observed nucleotides are possible:
    the cost of the observed nucleotide is zero, and
    the cost of any other nucleotide is infinity.
    """
    # YOUR CODE HERE
    raise NotImplementedError
    

The following code sets the sequence at the tips (from left to right) and initializes the tree using the function above. Set the sequence for your tree.

In [ ]:
tip_seq = 'CACAG'
my_tree = init_tree(len(tip_seq))

This code explicitly sets the tree topology. We can change the tree's structure by modifying these lines.

Note that the left and right nodes for the tips are left as `None` because they have no descendents.

In [ ]:
my_tree['root']['left'] = 'int1'
my_tree['root']['right'] = 'int3'
my_tree['int1']['left'] = 'tip1'
my_tree['int1']['right'] = 'tip2'
my_tree['int3']['left'] = 'tip3'
my_tree['int3']['right'] = 'int2'
my_tree['int2']['left'] = 'tip4'
my_tree['int2']['right'] = 'tip5'

In [ ]:
# View the dictionary representation of our tree
my_tree

This is what our tree would look like as a dendrogram:


<img src="https://github.com/melbournebioinformatics/COMP90014/blob/main/data/2023/Workshop_06/media/Example_tree_topology.png?raw=true" alt="Example_tree_topology.png" style="width: 500px;"/>

This loop uses the helpful built in function `enumerate` to initialize the tips of the tree using the function you wrote above. Double check that it is working correctly for the tip sequence you entered.

In [ ]:
for i, nt in enumerate(tip_seq):
    tip_name = 'tip{}'.format(i + 1)
    init_tip(my_tree, tip_name, nt)
    print(tip_name, my_tree[tip_name])

## Scoring trees

### Exercise 2: Sankoff function core calculation


<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

This function calculates the $S$ arrays for each node. It assumes that the left and right nodes have already been filled.

The outer loop is looping over the four entries (ACGT) in the $S$ array. 
    
<b>Challange:</b> Complete the function using the variables initialized and the calculation from lecture:

$S_a(i) = \min_{j}[c_{ij} + S_L(j)] + \min_{k}[c_{ik} + S_R(k)]$

As before, this function modifies the tree in place and doesn't return a value.
    
</div>

In [ ]:
def sankoff_calculate(c_matrix, tree, node_name):
    """
    For the specified node of the tree, calculate the minimum possible cost 
    for each nucleotide. 
    """
    # Find the min possible cost for moving to nucleotide nt1 in the current node 'node_name'
    for nt1 in CHARACTERS:
        left_costs = list()
        right_costs = list()
        left_child = tree[tree[node_name]['left']]
        right_child = tree[tree[node_name]['right']]
        
        # From each possible nucleotide 'nt2' of the left and right child nodes.
        # Use the initialized variables above to guide you.
        # Hint: We want to update tree[node_name][nt1] with the min cost value
        # YOUR CODE HERE
        raise NotImplementedError

Since the `sankoff_calculate` function requires that the left and right nodes be populated, we will perform a recursive traversal. 

In [ ]:
def sankoff_traverse(c_matrix, tree, node_name):
    """
    Traverse the tree recursively, calculating costs for each node.
    Calculate a node's children before the node itself.
    If a child node already has its costs calculated, don't try to calculate it.
    This function depends on correctly initialising the costs in the tips
    of the tree before traversing the tree.
    """
    print("Examining node {}".format(node_name))
    if tree[tree[node_name]['left']]['A'] is None:
        sankoff_traverse(c_matrix, tree, tree[node_name]['left'])
    else:
        print("Node {} values already known".format(tree[node_name]['left']))
    
    if tree[tree[node_name]['right']]['A'] is None:
        sankoff_traverse(c_matrix, tree, tree[node_name]['right'])
    else:
        print("Node {} values already known".format(tree[node_name]['right']))   
    
    print("Calculating node {}".format(node_name))
    sankoff_calculate(c_matrix, tree, node_name)

In [ ]:
sankoff_traverse(c, my_tree, 'root')

Finally, we can print the tree dictionary and see the $S$ array values for each node.

In [ ]:
my_tree

### Exercise 3: Finding the minimum cost

<div style="color: rgb(27,94,32); background: rgb(200,230,201); border: solid 1px rgb(129,199,132); padding: 10px;">

By examining the root specifically, we can determine the minimum cost of the tree and infer the ancestral state at this position according to this tree. 
    
<b>Challange:</b> Write some code that prints out the minimum cost at the root and all nucleotides with that cost.

Real reconstructions of ancestral sequences will incorporate lots of additional information and not be so simple as this!
    
</div>

In [ ]:
from math import inf
min_nts = list()
min_cost = inf

# YOUR CODE HERE
raise NotImplementedError

print("Minimum cost is {} for {}".format(min_cost, " and ".join(min_nts)))